In [50]:
import pandas as pd
import numpy as np
from fbprophet import Prophet
import tensorflow as tf
import matplotlib
import os
import matplotlib.pyplot as plt
from datetime import datetime
tf.set_random_seed(77)

mockForecastDictionary = {}
realForecastDictionary = {}

def LearningModuleRunner(rawArrayDatas, processId, forecastDay,dayOrWeekOrMonth):
    #TODO make dayOrWeekOrMonth parameter
    dayOrWeekOrMonth=dayOrWeekOrMonth
    # options:
    # 'day', 'week', 'month'

    feature = 'DayOfWeek_WeekNumber_Month_Season'
    # options:
    # dayOrWeekOrMonth='day': 'DayOfWeek_WeekNumber_Month_Season','DayOfWeek01_WeekNumber_Month_Season'//
    # dayOrWeekOrMonth='week': 'WeekNumber_Month_Season_Year'

#     LoggingManager.PrintLogMessage("LearningManager", "LearningModuleRunner", "start of learning #" + str(processId), DefineManager.LOG_LEVEL_INFO)

    global mockForecastDictionary
    global realForecastDictionary
    mockForcastDay=2*forecastDay

    ##Make txsForRealForecastLstm   [:]
    ds = rawArrayDatas[0]
    y = list(rawArrayDatas[1])
    sales = list(zip(ds, y))
    txsForRealForecastLstm =pd.DataFrame(data=sales, columns=['ds', 'y'])

    ##Make txsForMockForecastLstm [:-forecastDay]
    ds = rawArrayDatas[0][:-forecastDay]
    y= list(rawArrayDatas[1][:-forecastDay] )
    sales = list(zip(ds, y))
    txsForMockForecastLstm =pd.DataFrame(data=sales, columns=['ds', 'y'])

    ##Make txsForRealForecastBayesian [:-forecastDay] & np.log
    ds = rawArrayDatas[0][:-forecastDay]
    # TODO bayseian에 대해서는 input값이 0인 상황처리 필요
    y = list(np.log(rawArrayDatas[1][:-forecastDay]))
    sales = list(zip(ds, y))
    txsForRealForecastBayesian = pd.DataFrame(data=sales, columns=['ds', 'y'])

    ##Make txsForMockForecastBayseian   [:-3*forecastDay] & np.log
    ds = rawArrayDatas[0][:-3*forecastDay]
    #TODO bayseian에 대해서는 input값이 0인 상황처리 필요
    y= list(np.log(rawArrayDatas[1][:-3*forecastDay]))
    sales = list(zip(ds, y))
    txsForMockForecastBayseian =pd.DataFrame(data=sales, columns=['ds', 'y'])

    #testY for algorithm compare has size of 2*forecastDay:  rawArrayDatas[1][-3*forecastDay:-forecastDay]
    testY= rawArrayDatas[1][-3*forecastDay:-forecastDay]


    if dayOrWeekOrMonth is 'day':
        ####LSTM_day

        #select feature module
        feature='DayOfWeek_WeekNumber_Month_Season'

        mockForecastDictionary['LSTM'] = LSTM(txsForMockForecastLstm, mockForcastDay,feature)

      
        ####Bayseian_day

        mockForecastDictionary['Bayseian'] = Bayseian(txsForMockForecastBayseian, mockForcastDay, 'day')

        #알고리즘 비교
        nameOfBestAlgorithm= AlgorithmCompare(testY)
        ####더 좋은 알고리즘 호출
        if nameOfBestAlgorithm is 'LSTM':
            tf.reset_default_graph()
            realForecastDictionary['LSTM'] = LSTM(txsForRealForecastLstm, forecastDay,feature)

        elif nameOfBestAlgorithm is 'Bayseian':
            realForecastDictionary['Bayseian']=Bayseian(txsForRealForecastBayesian,forecastDay,'day')

            
    elif dayOrWeekOrMonth is 'week':
        
        ####LSTM_week

        # select feature module
        feature = 'WeekNumber_Month_Season_Year'

        mockForecastDictionary['LSTM'] = LSTM(txsForMockForecastLstm, mockForcastDay, feature)


        ####Bayseian_week

        mockForecastDictionary['Bayseian'] = Bayseian(txsForMockForecastBayseian, mockForcastDay, 'week')


        # 알고리즘 비교
        nameOfBestAlgorithm = AlgorithmCompare(testY)
        ####더 좋은 알고리즘 호출
        if nameOfBestAlgorithm is 'LSTM':
            tf.reset_default_graph()
            realForecastDictionary['LSTM'] = LSTM(txsForRealForecastLstm, forecastDay,feature)

        elif nameOfBestAlgorithm is 'Bayseian':
     
            realForecastDictionary['Bayseian'] = Bayseian(txsForRealForecastBayesian, forecastDay, 'week')

            
    elif dayOrWeekOrMonth is 'month':
       
        ####LSTM_month

        # select feature module
        feature = 'Month_Season_Year'

        mockForecastDictionary['LSTM'] = LSTM(txsForMockForecastLstm, mockForcastDay, feature)    

        ####Bayseian_month

        mockForecastDictionary['Bayseian'] = Bayseian(txsForMockForecastBayseian, mockForcastDay, 'month')


        # 알고리즘 비교
        nameOfBestAlgorithm = AlgorithmCompare(testY)
        ####더 좋은 알고리즘 호출
        if nameOfBestAlgorithm is 'LSTM':
            tf.reset_default_graph()
            realForecastDictionary['LSTM'] = LSTM(txsForRealForecastLstm, forecastDay,feature)

        elif nameOfBestAlgorithm is 'Bayseian':
            realForecastDictionary['Bayseian'] = Bayseian(txsForRealForecastBayesian, forecastDay, 'month')

        ####################################################################################BAYSEIAN
    # tf.reset_default_graph()
    # realForecastDictionary['LSTM'] = LSTM(txsForRealForecastLstm, forecastDay, feature)

    data = rawArrayDatas[1][:-forecastDay] + realForecastDictionary[nameOfBestAlgorithm]
    date= rawArrayDatas[0]
    print("===================================THE END===================================================")
    return data


def LSTM(txs, forecastDay, features):
    tf.reset_default_graph()
    tf.set_random_seed(77)
    # Add basic date related features to the table
    year = lambda x: datetime.strptime(x, "%Y-%m-%d").year
    dayOfWeek = lambda x: datetime.strptime(x, "%Y-%m-%d").weekday()
    month = lambda x: datetime.strptime(x, "%Y-%m-%d").month
    weekNumber = lambda x: datetime.strptime(x, "%Y-%m-%d").strftime('%V')
    txs['year'] = txs['ds'].map(year)
    txs['month'] = txs['ds'].map(month)
    txs['weekNumber'] = txs['ds'].map(weekNumber)
    txs['dayOfWeek'] = txs['ds'].map(dayOfWeek)

    # Add non-basic date related features to the table
    seasons = [0, 0, 1, 1, 1, 2, 2, 2, 3, 3, 3, 0]  # dec - feb is winter, then spring, summer, fall etc
    season = lambda x: seasons[(datetime.strptime(x, "%Y-%m-%d").month - 1)]
    day_of_week01s = [0, 0, 0, 0, 0, 1, 1]
    day_of_week01 = lambda x: day_of_week01s[(datetime.strptime(x, "%Y-%m-%d").weekday())]
    txs['season'] = txs['ds'].map(season)
    txs['dayOfWeek01'] = txs['ds'].map(day_of_week01)

    # Backup originalSales
    originalSales = list(txs['y'])
    sales = list(txs['y'])
    
    #week number는 경계부분에서 약간 오류가 있다.
    if features is 'DayOfWeek_WeekNumber_Month_Season':
        tempxy = [list(txs['dayOfWeek']), list(txs['weekNumber']), list(txs['month']), list(txs['season']), sales]
    elif features is 'DayOfWeek01_WeekNumber_Month_Season':
        tempxy = [list(txs['dayOfWeek01']), list(txs['weekNumber']), list(txs['month']), list(txs['season']), sales]

    elif features is 'WeekNumber_Month_Season_Year':
        tempxy = [list(txs['weekNumber']), list(txs['month']), list(txs['season']), list(txs['year']), sales]
    
   
    xy = np.array(tempxy).transpose().astype(np.float)
    


    # Backup originalXY for denormalize
    originalXY = np.array(tempxy).transpose().astype(np.float)
    xy = minMaxNormalizer(xy)

    # TRAIN PARAMETERS
    # data_dim은 y값 도출을 위한 feature 가지수+1(독립변수 가지수 +1(y포함))
    data_dim = 5
    # data_dim크기의 data 한 묶음이 seq_length만큼 input으로 들어가
    seq_length = 10
    # output_dim(=forecastDays)만큼의 다음날 y_data를 예측
    output_dim = forecastDay
    # hidden_dim은 정말 임의로 설정
    hidden_dim = 100
    # learning rate은 배우는 속도(너무 크지도, 작지도 않게 설정)
    learning_rate = 0.001
    iterations=1000
    # Build a series dataset(seq_length에 해당하는 전날 X와 다음 forecastDays에 해당하는 Y)
    x = xy
    y = xy[:, [-1]]
    dataX = []
    dataY = []
    for i in range(0, len(y) - seq_length - forecastDay + 1):
        _x = x[i:i + seq_length]
        _y = y[i + seq_length:i + seq_length + forecastDay]
        _y = np.reshape(_y, (forecastDay))
        dataX.append(_x)
        dataY.append(_y)

    train_size = int(len(dataY) - forecastDay)
    # train_size = int(len(dataY) * 0.7)
    test_size = len(dataY) - train_size

    trainX, testX = np.array(dataX[0:train_size]), np.array(dataX[train_size:])

    trainY, testY = np.array(dataY[0:train_size]), np.array(dataY[train_size:])

    X = tf.placeholder(tf.float32, [None, seq_length, data_dim])
    Y = tf.placeholder(tf.float32, [None, forecastDay])

    cell = tf.contrib.rnn.BasicLSTMCell(num_units=hidden_dim, state_is_tuple=True, activation=tf.tanh)
    outputs, _states = tf.nn.dynamic_rnn(cell, X, dtype=tf.float32)
    Y_pred = tf.contrib.layers.fully_connected(outputs[:, -1], output_dim, activation_fn=None)
    loss = tf.reduce_sum(tf.square(Y_pred - Y))  # sum of the squares
    optimizer = tf.train.AdamOptimizer(learning_rate)
    train = optimizer.minimize(loss)

    denormalizedTestY = originalSales[train_size + seq_length:]
    #     denormalizedTestY_feed=np.array([[i] for i in denormalizedTestY])

    targets = tf.placeholder(tf.float32, [None, 1])
    predictions = tf.placeholder(tf.float32, [None, 1])

    count = 0
    with tf.Session() as sess:

        # 초기화
        init = tf.global_variables_initializer()
        sess.run(init)

        # Training step
        for i in range(iterations):

            _, step_loss = sess.run([train, loss], feed_dict={X: trainX, Y: trainY})
            print("[step: {}] loss: {}".format(i, step_loss))


        # Test step
        test_predict = minMaxDeNormalizer(sess.run(Y_pred, feed_dict={X: testX}), originalXY)
        realSale = minMaxDeNormalizer(testY[-1], originalXY)
    return test_predict[-1].tolist()

def Bayseian(txs, forecastDay, unit):
    global mockForecastDictionary
    global realForecastDictionary

    if unit is 'day':
        if (len(txs) < 366):
            model = Prophet()
            model.fit(txs)
            future = model.make_future_dataframe(periods=forecastDay)
            forecastProphetTable = model.predict(future)

        else:
            model = Prophet(yearly_seasonality=True)
            model.fit(txs)
            future = model.make_future_dataframe(periods=forecastDay)
            forecastProphetTable = model.predict(future)


    elif unit is 'week':
        if(len(txs)<53):
            model = Prophet()
            model.fit(txs)
            future = model.make_future_dataframe(periods=forecastDay,freq='w')
            forecastProphetTable = model.predict(future)

        else:
            model = Prophet(yearly_seasonality=True)
            model.fit(txs)
            future = model.make_future_dataframe(periods=forecastDay,freq='w')
            forecastProphetTable = model.predict(future)

    elif unit is 'month':
        if(len(txs)<12):
            model = Prophet()
            model.fit(txs)
            future = model.make_future_dataframe(periods=forecastDay,freq='m')
            forecastProphetTable = model.predict(future)

        else:
            model = Prophet(yearly_seasonality=True)
            model.fit(txs)
            future = model.make_future_dataframe(periods=forecastDay,freq='m')
            forecastProphetTable = model.predict(future)


    #date = [d.strftime('%Y-%m-%d') for d in forecastProphetTable['ds']]
    return [np.exp(y) for y in forecastProphetTable['yhat'][-forecastDay:]]

def rmse(a,b):
    sum=0
    for i in range(len(a)):
        sum=sum+(a[i]-b[i])**2
    return np.sqrt(sum/len(a))

def minMaxNormalizer(data):
    numerator=data-np.min(data)
    denominator=np.max(data)-np.min(data)
    return numerator/(denominator+1e-7)

def minMaxDeNormalizer(data, originalData):
    shift=np.min(originalData)
    multiplier=np.max(originalData)-np.min(originalData)
    return (data+shift)*multiplier

def AlgorithmCompare(testY):
    global mockForecastDictionary
    nameOfBestAlgorithm = 'LSTM'
    minData = rmse(testY, mockForecastDictionary[nameOfBestAlgorithm])
    rms = 0
    for algorithm in mockForecastDictionary.keys():
        rms = rmse(testY, mockForecastDictionary[algorithm])
        if rms < minData:
            nameOfBestAlgorithm = algorithm
    print('testY is: ', testY)
    print(mockForecastDictionary['LSTM'])
    print('LSTM forecast :',mockForecastDictionary['LSTM'] , '@@@@@LSTM rmse: ',rmse(testY, mockForecastDictionary['LSTM']) )
    print('Bayseian forecast :',mockForecastDictionary['Bayseian'], '@@@@@Bayseian rmse: ',rmse(testY, mockForecastDictionary['Bayseian']) ) 
    print(nameOfBestAlgorithm, 'WON!!!!!!')
    return nameOfBestAlgorithm



In [51]:
# def LSTM(txs, forecastDay, features)
# def Bayseian(txs, forecastDay, unit)

In [52]:
# columns=['ds','y']
# txs=pd.read_table('walMonth_train.csv', sep=',',header=None,names=columns )

In [53]:
# len(txs)
# LSTM(txs, 7, features)
# Bayseian(txs, 7, 'month')

In [54]:
rawArrayDatas=[ ['2007-12-01', '2008-01-01', '2008-02-01', '2008-03-01', '2008-04-01', '2008-05-01', '2008-06-01', '2008-07-01', '2008-08-01', '2008-09-01', '2008-10-01', '2008-11-01', '2008-12-01', '2009-01-01', '2009-02-01', '2009-03-01', '2009-04-01', '2009-05-01', '2009-06-01', '2009-07-01', '2009-08-01', '2009-09-01', '2009-10-01', '2009-11-01', '2009-12-01', '2010-01-01', '2010-02-01', '2010-03-01', '2010-04-01', '2010-05-01', '2010-06-01', '2010-07-01', '2010-08-01', '2010-09-01', '2010-10-01', '2010-11-01', '2010-12-01', '2011-01-01', '2011-02-01', '2011-03-01', '2011-04-01', '2011-05-01', '2011-06-01', '2011-07-01', '2011-08-01', '2011-09-01', '2011-10-01', '2011-11-01', '2011-12-01', '2012-01-01', '2012-02-01', '2012-03-01', '2012-04-01', '2012-05-01', '2012-06-01', '2012-07-01', '2012-08-01', '2012-09-01', '2012-10-01', '2012-11-01', '2012-12-01', '2013-01-01', '2013-02-01', '2013-03-01', '2013-04-01', '2013-05-01', '2013-06-01', '2013-07-01', '2013-08-01', '2013-09-01', '2013-10-01', '2013-11-01', '2013-12-01', '2014-01-01', '2014-02-01', '2014-03-01', '2014-04-01', '2014-05-01', '2014-06-01', '2014-07-01', '2014-08-01', '2014-09-01', '2014-10-01', '2014-11-01', '2014-12-01', '2015-01-01', '2015-02-01', '2015-03-01', '2015-04-01', '2015-05-01', '2015-06-01', '2015-07-01', '2015-08-01', '2015-09-01', '2015-10-01', '2015-11-01', '2015-12-01', '2016-01-01'] , [4958.0, 8359.2999999999993, 12178.14286, 1735.5357140000001, 2178.5666670000001, 1424.419355, 1237.7142859999999, 1343.4545449999998, 2100.580645, 3135.5999999999999, 2644.517241, 2929.0, 2971.4838709999999, 4590.8709680000002, 2453.4642859999999, 1603.2903229999999, 1760.5, 1483.4838710000001, 1376.5333330000001, 1725.9354840000001, 2844.6451609999999, 5786.4615380000005, 6445.6428569999998, 6952.2142860000004, 7556.0322580000002, 13760.172409999999, 12469.23077, 2277.3225809999999, 2503.9666670000001, 1698.451613, 1425.5357140000001, 1848.6153850000001, 3838.3225810000004, 9605.4666670000006, 6985.6774189999996, 8732.5, 7899.0645159999995, 9540.9032260000004, 3907.1071430000002, 2085.8709680000002, 3015.5666670000001, 1886.193548, 1695.9000000000001, 3470.6451609999999, 4622.2258060000004, 9086.6896550000001, 6864.8666670000002, 7461.9666670000006, 7556.3103449999999, 19610.45161, 25008.793099999999, 24771.03226, 3943.3103450000003, 3136.3870969999998, 2406.8666670000002, 3127.6451609999999, 5017.7741939999996, 13273.733329999999, 9823.8064519999989, 7519.6666670000004, 9169.16129, 13726.903230000002, 4432.5, 2776.5161290000001, 2286.6333329999998, 1913.451613, 1650.9333329999999, 2040.0333329999999, 2734.0645159999999, 13740.266669999999, 10564.35484, 7608.3999999999996, 8706.1935480000011, 26366.56667, 24504.89286, 2169.0322579999997, 1879.0, 2449.5483870000003, 1487.9000000000001, 1903.580645, 3121.333333, 8773.6333329999998, 9429.2903230000011, 4370.5666670000001, 4122.7096770000007, 7644.7419349999991, 4147.0740740000001, 1799.7096770000001, 1405.5, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0] ]

In [55]:
forecastDay=int(len(rawArrayDatas[1])*0.1)

In [56]:
forecastDay

9

In [57]:
answer=LearningModuleRunner(rawArrayDatas,7,forecastDay,'month')

UnboundLocalError: local variable 'tempxy' referenced before assignment

In [ ]:
a=[6235.7822265625, 6553.90087890625, 6649.3369140625, 4946.8583984375, 8058.6650390625, 6564.55029296875, 1695.83251953125, 2567.910400390625, 6391.9208984375, 7342.6611328125, 5826.0869140625, 5460.5498046875, 7682.72607421875, 18227.732421875, 23328.7109375, 24845.134765625, 3408.103515625, 4330.37109375]

In [19]:
len(a)

18